NOTE: The github viewer doesnt support embeded svg in img tags at this time. To view this notebook correctly, use
an external viewer. You can view it with vscode as well.

Here is the nbviewer link
https://nbviewer.org/github/tiby312/poloto-project/blob/master/poloto-evcxr/example.ipynb

Alternative
https://jsvine.github.io/nbpreview/.

First lets import poloto.

In [2]:
:dep poloto

Next import evcxr rendering for poloto.

In [3]:
:dep poloto-evcxr
use poloto_evcxr::prelude::*;

Now lets make a simple function that just displays one plot to avoid having to write alot of boilerplate.

In [4]:
// import hypermelon crate so that we can manually append elements.
:dep hypermelon
use hypermelon::prelude::*;
    
fn show(it:impl IntoIterator<Item=[f64;2]>){
    let j=poloto::build::plot("a").line(it.into_iter());
    let svg = poloto::header().with_dim([400.0,300.0]);
    let style = poloto::render::Theme::light();
    let res=poloto::frame()
        .with_tick_lines([true, true])
        .build()
        .data(j)
        .build_and_label(("hello","x","y"));
    res.append_to(svg.append(style)).render_evcxr_img();
}

Ok now lets plot some data.

In [5]:
show(vec!([0.0,0.0],[1.0,1.0],[3.0,5.0]));

Ok cool. Now lets change the data.

In [6]:
show(vec!([1.0,5.0],[2.0,1.0],[3.0,0.0]));

Oh no now the data is going down!

Here is an gaussian example

In [7]:
{
    use poloto::build;

    // See https://en.wikipedia.org/wiki/Gaussian_function
    let gau = |sigma: f64, mu: f64| {
        use std::f64::consts::TAU;
        let s = sigma.powi(2);
        let k = (sigma * TAU).sqrt().recip();
        move |x: f64| [x, (-0.5 * (x - mu).powi(2) / s).exp() * k]
    };

    let xs = poloto::util::range_iter([-5.0, 5.0], 200);

    let plots = poloto::plots!(
        build::plot("σ=1.0").line(xs.clone().map(gau(1.0, 0.0))),
        build::plot("σ=0.5").line(xs.clone().map(gau(0.5, 0.0))),
        build::plot("σ=0.3").line(xs.clone().map(gau(0.3, 0.0)))
    );

    poloto::frame_build()
        .data(poloto::plots!(build::origin(), plots))
        .build_and_label(("gaussian", "x", "y"))
        .append_to(poloto::header().light_theme())
        .render_evcxr_img();
}

()

Here is a collatz example

In [8]:
{
    use poloto::build;

    let collatz = |mut a: i128| {
    std::iter::from_fn(move || {
            if a == 1 {
                None
            } else {
                a = if a % 2 == 0 { a / 2 } else { 3 * a + 1 };
                Some(a)
            }
        })
        .fuse()
    };

    let svg = poloto::header().with_viewbox_width(1200.0);

    let style =
        poloto::render::Theme::dark().append(".poloto_line{stroke-dasharray:2;stroke-width:2;}");

    let a = (1000..1006).map(|i| build::plot(format!("c({})", i)).line((0..).zip(collatz(i))));

    poloto::frame()
        .with_tick_lines([true, true])
        .with_viewbox(svg.get_viewbox())
        .build()
        .data(poloto::plots!(poloto::build::origin(), a))
        .build_and_label(("collatz", "x", "y"))
        .append_to(svg.append(style))
        .render_evcxr_img();
}

()